# Geração de dados
## Os dados produzidos serão utilizados para avaliar os modelos de regressão simbólica

In [2]:
import numpy as np
from scipy.integrate import solve_ivp
import pandas as pd
# data = np.loadtxt("/mnt/c/Users/beatr/Desktop/GPLearn/gplearn/doc/dados_sirI.txt")
# np.savetxt('dados_sirR.txt', data)

In [3]:
def sir(t, x, beta=0.3, gamma=0.1):
    return [
        -beta * x[0] * x[1],
        beta * x[0] * x[1] - gamma * x[1],
        gamma * x[1],
    ]


t_start = 0.0
t_end = 100.0
t_step =  0.002

t_span = (t_start, t_end)

t_eval = np.arange(t_start, t_end, t_step)   # 
# Criando o vetor de condições iniciais
N = 1
I0, R0 = 0.001, 0
S0 = N - I0 - R0
x0_train = np.array([S0, I0, R0]) 

solution = solve_ivp(sir, t_span, x0_train,  method='RK45', t_eval=t_eval)

t = solution.t
pop = solution.y
S = pop[:, 0]
I =pop[:, 1]
R = pop[:, 2]

In [4]:
# Simule as derivadas

def derivadas(S, I, R, beta, gamma):
    # Coeficientes das derivadas
    dSdt = -beta * S * I 
    dIdt =  beta * S * I - gamma * I
    dRdt = gamma * I
    return [dSdt, dIdt, dRdt]

# Parâmetros do modelo
beta = 0.3 # Coeficiente de transmissão
gamma = 0.1  # Taxa de recuperação

# Calcular as derivadas
dSdt, dIdt, dRdt = derivadas(S, I, R, beta, gamma)


# Derivadas numéricas usando np.gradient
#dSdt = np.gradient(solution.y[0], solution.t)
#dIdt = np.gradient(solution.y[1], solution.t)
#dRdt = np.gradient(solution.y[2], solution.t)


# Resultados
#print("dS/dt:", dSdt)
#print("dI/dt:", dIdt)
#print("dR/dt:", dRdt)
                                       
df = pd.DataFrame({'suscetiveis': S, 'infectados': I, 'recuperado': R, 'dSdt': dSdt,
                                                                        'dIdt': dIdt, 
                                                                        'dRdt': dRdt })  
df.head()
# Salvando o DataFrame em um arquivo CSV
df.to_csv('dados_sir.csv', index=False)
# Dados originais com suas taxas de variação dentro da pasta S_sir
df.to_csv('dados_sir.txt', sep='\t', index=False)
###############################

df_S = pd.DataFrame({'suscetiveis': S, 'infectados': I,'dSdt': dSdt}) 
np.savetxt('dados_sir_S.txt', df_S)
df_I = pd.DataFrame({'suscetiveis': S, 'infectados': I,'dSdt': dIdt}) 
np.savetxt('dados_sir_I.txt', df_I)
df_R = pd.DataFrame({'infectados': I,'dRdt': dRdt}) 
np.savetxt('dados_sir_R.txt', df_R)